### Mengimport Library yang diperlukan

In [206]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

### Membaca File CSV

In [207]:
df = pd.read_csv("netflix daily top 10 bersih.csv" , sep = ";")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            7100 non-null   int64  
 1   As of                 7100 non-null   object 
 2   Rank                  7100 non-null   int64  
 3   Year to Date Rank     7100 non-null   float64
 4   Last Week Rank        7100 non-null   float64
 5   Title                 7100 non-null   object 
 6   Type                  7100 non-null   object 
 7   Netflix Exclusive     7100 non-null   object 
 8   Netflix Release Date  7100 non-null   object 
 9   Days In Top 10        7100 non-null   int64  
 10  Viewership Score      7100 non-null   int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 610.3+ KB


In [208]:
df.head()

,Unnamed: 0,As of,Rank,Year to Date Rank,Last Week Rank,Title,Type,Netflix Exclusive,Netflix Release Date,Days In Top 10,Viewership Score
0,0,01/04/2020,1,1.0,1.000000,"Tiger King: Murder, Mayhem …",TV Show,Yes,"Mar 20, 2020",9,90
1,1,01/04/2020,2,2.0,3.938378,Ozark,TV Show,Yes,"Jul 21, 2017",5,45
2,2,01/04/2020,3,3.0,2.000000,All American,TV Show,Yes,"Mar 28, 2019",9,76
3,3,01/04/2020,4,4.0,3.938378,Blood Father,Movie,Yes,"Mar 26, 2020",5,30
4,4,01/04/2020,5,5.0,4.000000,The Platform,Movie,Yes,"Mar 20, 2020",9,55


### Mengecek Nilai Null

In [209]:
df.isna().sum()

Unnamed: 0              0
As of                   0
Rank                    0
Year to Date Rank       0
Last Week Rank          0
Title                   0
Type                    0
Netflix Exclusive       0
Netflix Release Date    0
Days In Top 10          0
Viewership Score        0
dtype: int64

### Mengecek Nilai Duplikat

In [210]:
df.duplicated().sum()

0

### Mengecek Nilai Unik pada Attribut Type untuk menentukan class

In [195]:
df['Type'].unique()

array(['TV Show', 'Movie', 'StandUp Comedy', 'Concert/Perf…'],
      dtype=object)

### Mengubah Data Objek (Categorical) dengan Ordinal Encoder

In [211]:
encoder = OrdinalEncoder()
df["As of"] = encoder.fit_transform(df[["As of"]])
df["Title"] = encoder.fit_transform(df[["Title"]])
df["Type"] = encoder.fit_transform(df[["Type"]])
df["Netflix Exclusive"] = encoder.fit_transform(df[["Netflix Exclusive"]])
df["Netflix Release Date"] = encoder.fit_transform(df[["Netflix Release Date"]])

### Data Splitting

In [212]:
X = df[['As of', 'Rank', 'Year to Date Rank', 
        'Last Week Rank', 'Title', 'Netflix Exclusive',
        'Netflix Release Date', 'Days In Top 10', 'Viewership Score']]
Y = df['Type']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)

# Menampilkan Dimensi
print("Dimensi X_Train: ", X_train.shape)
print("Dimensi X_Test: ", X_test.shape)
print("Dimensi Y_Train: ", Y_train.shape)
print("Dimensi Y_Test: ", Y_test.shape)

Dimensi X_Train:  (4970, 9)
Dimensi X_Test:  (2130, 9)
Dimensi Y_Train:  (4970,)
Dimensi Y_Test:  (2130,)


### Encoding Label

In [213]:
num_class = 4

y_train = tf.keras.utils.to_categorical(Y_train, num_class)
y_test = tf.keras.utils.to_categorical(Y_test, num_class)

### Membuat Deep Neural Network dengan Sequential Model

In [214]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(512, activation='relu', input_dim = 9),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')
    ]
)
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 512)               5120      
                                                                 
 dense_41 (Dense)            (None, 256)               131328    
                                                                 
 dense_42 (Dense)            (None, 128)               32896     
                                                                 
 dense_43 (Dense)            (None, 4)                 516       
                                                                 
Total params: 169,860
Trainable params: 169,860
Non-trainable params: 0
_________________________________________________________________


### Optimisasi Model

In [219]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

### Fitting Training Model

In [220]:
model.fit(X_train, y_train, epochs = 10, batch_size = 512, validation_data = (X_test, y_test))

Epoch 1/10
10/10 [==============================] - 1s 49ms/step - loss: 0.3305 - accuracy: 0.6467 - val_loss: 0.3005 - val_accuracy: 0.6568
Epoch 2/10
10/10 [==============================] - 0s 23ms/step - loss: 0.2787 - accuracy: 0.6746 - val_loss: 0.2840 - val_accuracy: 0.6634
Epoch 3/10
10/10 [==============================] - 0s 26ms/step - loss: 0.2673 - accuracy: 0.6877 - val_loss: 0.2771 - val_accuracy: 0.6723
Epoch 4/10
10/10 [==============================] - 0s 31ms/step - loss: 0.2626 - accuracy: 0.6966 - val_loss: 0.2751 - val_accuracy: 0.6911
Epoch 5/10
10/10 [==============================] - 0s 24ms/step - loss: 0.2616 - accuracy: 0.7012 - val_loss: 0.2784 - val_accuracy: 0.6793
Epoch 6/10
10/10 [==============================] - 0s 24ms/step - loss: 0.2571 - accuracy: 0.7135 - val_loss: 0.2738 - val_accuracy: 0.6840
Epoch 7/10
10/10 [==============================] - 0s 28ms/step - loss: 0.2569 - accuracy: 0.7087 - val_loss: 0.2767 - val_accuracy: 0.7042
Epoch 8/10
10

### Evaluasi Model yang telah ditrain

#### Akurasi Score Testing

In [221]:
score_testing = model.evaluate(X_test, y_test)
print("TEST LOSS      : ", score_testing[0])
print("TEST ACCURACY  : ", score_testing[1])

67/67 [==============================] - 0s 3ms/step - loss: 0.2751 - accuracy: 0.7052
TEST LOSS      :  0.27505776286125183
TEST ACCURACY  :  0.7051643133163452


#### Akurasi Score Training

In [222]:
score_training = model.evaluate(X_train, y_train)
print("TEST LOSS      : ", score_training[0])
print("TEST ACCURACY  : ", score_training[1])

156/156 [==============================] - 0s 3ms/step - loss: 0.2456 - accuracy: 0.7320
TEST LOSS      :  0.24555647373199463
TEST ACCURACY  :  0.7319919466972351


### Simpan Model

In [223]:
model.save('Hasil_Posttest8.h5')